# Convex Hull Continuum Removal

This notebook reads the hyperspectral mosaic, calculates a convex hull and outputs a convex hull continuum removed data set.

In [1]:
# import standard libaries
import os, warnings, subprocess
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt

# import gis libraries
import rasterio

# import convex hull module from Scipy
from scipy.spatial import ConvexHull

# filter warnings for tidy output
warnings.filterwarnings("ignore")

## 1. Run Convex Hull Removal

This is a pretty slow workflow that runs a pixel-wise convex hull removal processing step inspired by Ben Chi's [*Reading TSG Files*](https://www.fractalgeoanalytics.com/articles/reading-tsg-files/) blogpost.

In [3]:
# set path to mosaic ENVI grid
hsi_fn = r'/storage/gsq_hymap/processed/GSQ_block-d_mosaic_ref.dat'

# set path to output convex hull continuum removed ENVI grid
chull_fn = r'/storage/gsq_hymap/outputs/GSQ_block-d_mosaic_hull.dat'

# open the ENVI
with rasterio.open(hsi_fn,'r') as src:
  # create a metadata file for the output
  meta = src.meta.copy()
  meta.update({'dtype':'int16', 'driver':'ENVI', 'nodata':-999})
  # get a numpy array of the wavelegnths for each band, use band descriptions
  wvl = np.array([float(x.strip('nm')) for x in src.descriptions])
  # hymap spectra have a weird deep feature at 1390.4nm (band 62) that can interfere with the hull
  mask_b62 = wvl != 1390.4 # create a boolean mask to identify the offending band for later 
  # open the output file
  with rasterio.open(chull_fn,'w',**meta) as dst:
    # iterate through the native blockwindows (rows in ENVI grids)
    blockwindows = [x for x in src.block_windows()]
    for ji, block in tqdm(blockwindows):
      hsi_window = src.read(window=block)[:,0,:].T    # reshape window to (Npixels, Nwavelengths)
      # iterate through pixels in blockwindow and perform convex hull continuum removal 
      cvxhul_window = hsi_window.copy()
      for i, pix in enumerate(hsi_window):
        if pix[0] != src.nodata: # only consider non-nodata pixels
          fpix = np.interp(wvl, wvl[mask_b62], pix[mask_b62])       # linear interpolation to fix weird band 62 values
          cvxhul = ConvexHull(np.vstack([fpix, wvl]).T)             # fit convex hull to fixed spectrum
          vtx = np.sort(cvxhul.vertices)                            # get sorted vertices of the hull
          hull = np.interp(np.arange(wvl.shape[0]), vtx, pix[vtx])  # linear interpolation of hull for all wavelengths 
          cvxhul_window[i] = pix - hull                             # remove hull from spectrum and place in output window
      # write the convex hull removed data to the output file
      dst_out = np.zeros((wvl.shape[0],1,cvxhul_window.shape[0]), dtype=cvxhul_window.dtype)
      dst_out[:,0,:] = cvxhul_window.T
      dst.write(dst_out, window=block)
    # write band descriptions
    for i, d in enumerate(src.descriptions):
      dst.set_band_description(i+1, src.descriptions[i])
    dst.close()
  src.close()

 18%|█▊        | 1279/7285 [09:27<1:27:52,  1.14it/s]

## 2. Validate Results

Plot a random selection of pixels from the raw and convex hull removed data cube to confirm the above has worked.

## 3. Calculate Calcite Abundance Index